# XOR_DNN

In [ ]:
import tensorflow as tf
import numpy as np

tf.random.set_seed(777)

learning_rate = 0.1
training_cnt = 10000

X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
Y = np.array([[0], [1], [1], [0]], dtype=np.float32)

# Layer 1
W1 = tf.Variable(tf.random.normal([2, 2]), name='weight1',dtype=tf.float32,trainable=True)
b1 = tf.Variable(tf.random.normal([2]), name='bias1', dtype=tf.float32, trainable=True)

# Layer 2
W2 = tf.Variable(tf.random.normal([2, 1]), name='weight2', dtype=tf.float32, trainable=True)
b2 = tf.Variable(tf.random.normal([1]), name='bias2', dtype=tf.float32, trainable=True)


for step in range(training_cnt):
    with tf.GradientTape() as tape:
        L1 = tf.sigmoid(tf.matmul(X, W1) + b1)
        pred = tf.sigmoid(tf.matmul(L1, W2) + b2)
        cost = -tf.reduce_mean(Y * tf.math.log(pred) + (1 - Y) * tf.math.log(1-pred))
    weight_list = [W1,W2,b1,b2]
    W1_grad, W2_grad, b1_grad, b2_grad = tape.gradient(cost, weight_list)
    W1.assign_sub(learning_rate*W1_grad)
    W2.assign_sub(learning_rate*W2_grad)
    b1.assign_sub(learning_rate*b1_grad)
    b2.assign_sub(learning_rate*b2_grad)
    predicted = tf.cast(pred > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))  
    if step % 1000 == 0:
        print(step, cost.numpy(), W1.numpy(), W2.numpy(), b1.numpy(), b2.numpy())

print("\nPred: ", pred.numpy(), "\nPredicted: ", predicted.numpy(), "\nAccuracy: ", accuracy.numpy())

#### tf.random.set_seed
- 랜덤하게 생성되는 숫자들을 동일하게 생성하기 위한 것으로 실습 결과 비교를 위해 실행한다 

In [ ]:
tf.random.set_seed(777)

#### 파라메터 값 설정
- 학습을 위한 기초 파라메터
- learning_rate : 값이 너무 적으면 Train 되지 않을 수 있고 값이 너무 크면 overshooting이 발생할 수 있다
- training_cnt : dataset에 대한 training 반복 횟수

In [ ]:
learning_rate = 0.1
training_cnt = 10000

#### 트레이닝 데이터 변수 선언
- 입력으로 들어가는 X(input 2개), Y(output 1개) 설정
- numpy array를 사용

In [ ]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
Y = np.array([[0], [1], [1], [0]], dtype=np.float32)

#### tf.random.normal
- bias, weight의 초기값을 난수로 생성

In [ ]:
W1 = tf.Variable(tf.random.normal([2, 2]), name='weight1',dtype=tf.float32,trainable=True)
b1 = tf.Variable(tf.random.normal([2]), name='bias1', dtype=tf.float32, trainable=True)

#### sigmoid 함수 사용
- Y값이 0 또는 1의 binary 값을 갖기 때문에 logistic regression을 활용하고 sigmoid 함수를 사용한다

In [ ]:
L1 = tf.sigmoid(tf.matmul(X, W1) + b1)

#### 은닉층(hidden layer) 만들기
- "Layer 2"에선 첫 번째 레이어의 출력값인 "L1" 입력값이 된다
- 마지막 레이어에선 Weight 와 bias 값이 1개가 되도록 설정하고 sigmoid 함수를 사용하여 모델을 정의한다

In [ ]:
W2 = tf.Variable(tf.random.normal([2, 1]), name='weight2', dtype=tf.float32, trainable=True)
b2 = tf.Variable(tf.random.normal([1]), name='bias2', dtype=tf.float32, trainable=True)
pred = tf.sigmoid(tf.matmul(L1, W2) + b2)

#### cost/loss function 구현
- 0~1 사이의 값을 근사화 하기 위해서 log함수를 사용
$$C(H(x),y) = \frac{1}{m}∑(-y\log(H(x))-(1-y)\log(1-H(x))$$

In [ ]:
cost = -tf.reduce_mean(Y * tf.math.log(pred) + (1 - Y) * tf.math.log(1-pred))

#### 학습 방법 -> cost를 최소화 
- GradientDescent 함수 사용 (경사하강법)

In [ ]:
with tf.GradientTape() as tape:
    L1 = tf.sigmoid(tf.matmul(X, W1) + b1)
    pred = tf.sigmoid(tf.matmul(L1, W2) + b2)
    cost = -tf.reduce_mean(Y * tf.math.log(pred) + (1 - Y) * tf.math.log(1-pred))
weight_list = [W1,W2,b1,b2]
W1_grad, W2_grad, b1_grad, b2_grad = tape.gradient(cost, weight_list)
W1.assign_sub(learning_rate*W1_grad)
W2.assign_sub(learning_rate*W2_grad)
b1.assign_sub(learning_rate*b1_grad)
b2.assign_sub(learning_rate*b2_grad)

#### 학습된 예측값을 0과 1로 변환
- 0~1 사이로 학습된 예측값을 0과 1로 나누어 분류

In [ ]:
predicted = tf.cast(pred > 0.5, dtype=tf.float32)

#### 정확도 
- accuracy를 계산하여 분류가 정확한지 확인
- 예측값과 실제 데이터의 일치 여부 계산
- 아래 코드는 평균을 이용한 정확도 계산 

In [ ]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

#### 모델실행 
- pred는 sigmoid 함수를 통해 0~1 사이의 값으로 나온다
- predicted는 pred에서 나온 값을 0과 1로 변환 시킨 값이다
- accuracy는 '예측한 Y값과 실제 Y값과 얼마나 일치하는가'이다. 

In [ ]:
for step in range(training_cnt):
    with tf.GradientTape() as tape:
        L1 = tf.sigmoid(tf.matmul(X, W1) + b1)
        pred = tf.sigmoid(tf.matmul(L1, W2) + b2)
        cost = -tf.reduce_mean(Y * tf.math.log(pred) + (1 - Y) * tf.math.log(1-pred))
    weight_list = [W1,W2,b1,b2]
    W1_grad, W2_grad, b1_grad, b2_grad = tape.gradient(cost, weight_list)
    W1.assign_sub(learning_rate*W1_grad)
    W2.assign_sub(learning_rate*W2_grad)
    b1.assign_sub(learning_rate*b1_grad)
    b2.assign_sub(learning_rate*b2_grad)
    predicted = tf.cast(pred > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))  
    if step % 1000 == 0:
        print(step, cost.numpy(), W1.numpy(), W2.numpy(), b1.numpy(), b2.numpy())

print("\nPred: ", pred.numpy(), "\nPredicted: ", predicted.numpy(), "\nAccuracy: ", accuracy.numpy())